In [1]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
import os
import logging
import configparser

In [2]:
spark = SparkSession.builder \
    .appName("appGlob") \
    .config("spark.jars","../controller/postgresql-42.7.2.jar")\
    .config('spark.driver.extraClassPath', '../controller/postgresql-42.7.2.jar') \
    .getOrCreate()

In [3]:
config = configparser.ConfigParser()
config.read("../properties.conf")
database_config = config["database"]


KeyError: 'database'

In [ ]:
#log
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("ReadCSV")


In [ ]:
#LOAD_FILE
filePath = "../inputs/"
fileType = "departments.csv"
# Define the schema
dp_schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("department", StringType(), True)
])


In [ ]:
try: 
    df = spark.read.csv(filePath+fileType,header=False, inferSchema=False, schema=dp_schema ,sep=",")
    
except Exception as e:
    logger.error("Error: %s", str(e), exc_info=True)
    df = spark.createDataFrame([], schema=schema)

In [ ]:
logger.info("Dataframe schema for department table:")
df.printSchema()

In [ ]:
df.count()

In [ ]:
try :
    df.write \
    .format("jdbc") \
    .mode("overwrite") \
    .option("url", database_config["url"]) \
    .option("dbtable", "departments") \
    .option("user", database_config["user"]) \
    .option("password", database_config["password"]) \
    .option("driver", database_config["driver"]) \
    .save()
except Exception as e:
    logger.error("Erro: review eschemas or datatypes %s", str(e), exc_info=True)